In [0]:

base_path = "/Volumes/workspace/default/bigdataproject/"


path_names = f"{base_path}name.basics.tsv"
path_akas = f"{base_path}title.akas.tsv"
path_basics = f"{base_path}title.basics.tsv"
path_crew = f"{base_path}title.crew.tsv"
path_episode = f"{base_path}title.episode.tsv"
path_principals = f"{base_path}title.principals.tsv"
path_ratings = f"{base_path}title.ratings.tsv"

# Question 2 : How many total people in data set ? 

In [0]:

df_names = spark.read.format("csv") \
    .option("header", "true") \
    .option("sep", "\t") \
    .option("inferSchema", "true") \
    .option("nullValue", "\\N") \
    .load(path_names)

# We count the rows
total_people = df_names.count()

print(f"Total number of people in the Dataset : {total_people}")

# Question 3 & 7 :  What is the earliest year of birth? 
#                   What is the most recent date of birth?

In [0]:
from pyspark.sql.functions import min, max, col, current_date, year

stats = df_names.select(
    min(col("birthYear")).alias("earliest"),
    max(col("birthYear")).alias("most_recent")
).first()

earliest_year = int(stats["earliest"])
most_recent_year = int(stats["most_recent"])

print(f"Earliest birth year: {earliest_year}")
print(f"Most recent birth year: {most_recent_year}")



# Question 4 : How many years ago was this person born?

In [0]:
years_ago = 2025 - earliest_year
print(f"This person was born {years_ago} years ago.")


# Question 5 : Using only the data in the data set, determine if this date of birth correct.

In [0]:
print("\nThe person that was born the earliest :")
df_names.filter(col("birthYear") == earliest_year).show(truncate=False)

# Question 6 : Explain the reasoning for the answer in a code comment or new markdown cell. 

Basically, we searched in the dataframe the row where the "birthYear" is equal to the "earliest_year' that we found, and then we display the entire row.

Obviously, it is an outlier, because nobody should be born that long time ago, in that dataset

In [0]:
from pyspark.sql.functions import col

df_basics = spark.read.format("csv") \
    .option("header", "true") \
    .option("sep", "\t") \
    .option("nullValue", "\\N") \
    .load(path_basics) \
    .withColumn("runtimeMinutes", col("runtimeMinutes").cast("int")) \
    .withColumn("startYear", col("startYear").cast("int"))

df_basics.show(10)


# Question 8 : What percentage of the people do not have a listed date of birth?

In [0]:
# We check how many people there are in total in the dataset, and then how many missing values, and we can easily get the percentage of missing vlaues
total_people = df_names.count()
missing_birth_count = df_names.filter(col("birthYear").isNull()).count()

percentage_missing = (missing_birth_count / total_people) * 100

print(f"Percentage of people with no birth year: {percentage_missing:.2f}%")

# Question 9 : What is the length of the longest "short" after 1900?

In [0]:
from pyspark.sql.functions import max
# We filter for shorts, after 1900, and then find max runtime
longest_short = df_basics.filter(
    (col("titleType") == "short") & 
    (col("startYear") > 1900)
).select(max("runtimeMinutes")).first()[0]

print(f"Longest short film after 1900: {longest_short} minutes")

# Question 10 : What is the length of the shortest "movie" after 1900?

In [0]:
from pyspark.sql.functions import min

# We first filter for movies, after 1900, and then find min runtime
shortest_movie = df_basics.filter(
    (col("titleType") == "movie") & 
    (col("startYear") > 1900)
).select(min("runtimeMinutes")).first()[0]

print(f"Shortest movie after 1900: {shortest_movie} minutes")

# Question 11 : List of all of the genres represented.

In [0]:
from pyspark.sql.functions import split, explode

# We split the genres string into a list like for example ["Action", "Comedy"]
# We then get distinct values and sort them
unique_genres = df_basics.select(explode(split(col("genres"), ",")).alias("genre")) \
    .select("genre") \
    .distinct() \
    .sort("genre")

display(unique_genres)

# Question 12 : What is the highest rated comedy "movie"?

In [0]:
from pyspark.sql.functions import col, desc

# We load the Ratings data
df_ratings = spark.read.format("csv") \
    .option("header", "true") \
    .option("sep", "\t") \
    .option("nullValue", "\\N") \
    .load(path_ratings) \
    .withColumn("averageRating", col("averageRating").cast("double")) \
    .withColumn("numVotes", col("numVotes").cast("int"))

# We join basics (Movies) with Ratings
# We filter for 'movie' type and 'Comedy' genre
# We then sort by rating and votes
best_comedy = df_basics.join(df_ratings, "tconst") \
    .filter(col("titleType") == "movie") \
    .filter(col("genres").contains("Comedy")) \
    .orderBy(col("averageRating").desc(), col("numVotes").desc()) \
    .first()

best_comedy_id = best_comedy['tconst']

print(f"Title: {best_comedy['primaryTitle']}")
print(f"Rating: {best_comedy['averageRating']}")
print(f"Votes: {best_comedy['numVotes']}")
print(f"ID: {best_comedy_id}")

# Question 13 : Who was the director of the movie?

In [0]:
# We load Crew and Names data
df_crew = spark.read.format("csv") \
    .option("header", "true") \
    .option("sep", "\t") \
    .option("nullValue", "\\N") \
    .load(path_crew)

df_names = spark.read.format("csv") \
    .option("header", "true") \
    .option("sep", "\t") \
    .option("nullValue", "\\N") \
    .load(path_names)

# Director's ID
director_row = df_crew.filter(col("tconst") == best_comedy_id).first()
director_ids = director_row['directors']

if director_ids:
    # There can be multiple directors for a same movie, that is why we create a list
    ids_list = director_ids.split(",")
    
    print(f"Director ID(s): {ids_list}")
    
    # we search these ids in the names table
    directors = df_names.filter(col("nconst").isin(ids_list)) \
                        .select("primaryName") \
                        .collect()
    
    print("Director(s):")
    for row in directors:
        print(f"- {row['primaryName']}")
else:
    print("No director listed.")

# Question 14 : List, if any, the alternate titles for the movie.

In [0]:
# We load Alternate Titles data
df_akas = spark.read.format("csv") \
    .option("header", "true") \
    .option("sep", "\t") \
    .option("nullValue", "\\N") \
    .load(path_akas)

print(f"Alternate titles for '{best_comedy['primaryTitle']}':")

df_akas.filter(col("titleId") == best_comedy_id) \
    .select("title", "region", "language") \
    .show(truncate=False)